In [1]:
from torchvision import datasets, transforms
transform = transforms.ToTensor()
train_ds = datasets.FashionMNIST("data", train=True,  download=True, transform=transform)
test_ds  = datasets.FashionMNIST("data", train=False, download=True, transform=transform)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [2]:
from torch.utils.data import DataLoader
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl  = DataLoader(test_ds , batch_size=256)


In [5]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),               # 28×28 → 784
            nn.Linear(784, 256), nn.ReLU(),
            nn.Linear(256,128),  nn.ReLU(),
            nn.Linear(128, 10)          # logits
        )
    def forward(self, x): return self.net(x)

device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
          else "cpu")
model = MLP().to(device)

In [6]:
import torch, torch.optim as optim
loss_fn  = nn.CrossEntropyLoss()
opt      = optim.SGD(model.parameters(), lr=0.1)

epochs = 5
for ep in range(epochs):
    model.train()
    correct, total_loss = 0, 0
    for X,y in train_dl:
        X,y = X.to(device), y.to(device)
        opt.zero_grad()
        logits = model(X)
        loss   = loss_fn(logits, y)
        loss.backward()
        opt.step()

        total_loss += loss.item()*X.size(0)
        correct    += (logits.argmax(1) == y).sum().item()

    train_acc = correct / len(train_ds)

    # quick test pass
    model.eval(); test_correct=0
    with torch.no_grad():
        for X,y in test_dl:
            X = X.to(device); y = y.to(device)
            test_correct += (model(X).argmax(1)==y).sum().item()
    test_acc = test_correct / len(test_ds)
    print(f"Epoch {ep+1}: loss {total_loss/len(train_ds):.3f} | "
          f"train {train_acc:.1%} | test {test_acc:.1%}")


Epoch 1: loss 0.685 | train 75.3% | test 75.7%
Epoch 2: loss 0.433 | train 84.2% | test 83.1%
Epoch 3: loss 0.384 | train 86.0% | test 84.3%
Epoch 4: loss 0.355 | train 86.9% | test 84.8%
Epoch 5: loss 0.332 | train 87.7% | test 86.6%
